# We are going to Run this code 2 times :

# One in order to optimize LSTM Network for the prices of Bitcoin only and one to optimize it for all the features

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math

from sklearn.preprocessing import normalize, MinMaxScaler
from sklearn.metrics import mean_squared_error

import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.layers.core import Dropout, Activation
from keras.utils import np_utils

from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform

In [ ]:
def data():
    """
    Data providing function:

    This function is separated from create_model() so that hyperopt
    won't reload data for each evaluation run.
    """
    
    def create_dataset(dataset, look_back=1):

        dataX, dataY = [], []
        for i in range(len(dataset)-look_back-1):

            #The target is always the next value. And the lookback are the previous prices
            a = dataset[i:(i+look_back), :]
            dataX.append(a)
            dataY.append(dataset[i + look_back, :][0])

        return np.array(dataX), np.array(dataY)

    df = pd.read_csv('fulldata.csv')
    #Drop np.nan
    df.dropna(how='any', inplace=True)

    #Make it array
    df = df[['Weighted_Price', 'Polarity_Textblob','Polarity_Vader','Count_of_Tweets','Count_Of_Possitive_Tweets']]
    
    dataset = df.values
    dataset = dataset.astype('float32')
    
    # normalize the dataset
    scaler = MinMaxScaler(feature_range=(0, 1))
    dataset = scaler.fit_transform(dataset)

    # split into train and test sets
    train_size = int(len(dataset) * 0.60)
    test_size = len(dataset) - train_size
    train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
    
    # reshape into X=t and Y=t+1
    look_back = 5
    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)

    trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], trainX.shape[2]))
    testX = np.reshape(testX, (testX.shape[0], testX.shape[1], testX.shape[2]))

    return trainX, trainY, testX, testY

In [ ]:
def create_model(trainX, trainY, testX, testY):
    
    callbacks = [

            keras.callbacks.EarlyStopping(monitor='loss',patience=3),
            keras.callbacks.ModelCheckpoint(filepath='forecasting_price_model.h5', monitor='val_loss',save_best_only=True),
            keras.callbacks.ReduceLROnPlateau(monitor='val_loss',factor=0.1,patience=1),
            keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=1, mode='auto')
        ]

    # Define Model
    model = Sequential()
    model.add(LSTM( {{choice([4, 16, 32, 64, 128])}}, input_shape=(look_back, trainX.shape[2]),
                   dropout = {{uniform(0, 1)}}, recurrent_dropout={{uniform(0, 1)}},return_sequences=True
                  ))
    
    model.add(LSTM({{choice([4, 16, 32, 64, 128])}}, dropout = {{uniform(0, 1)}}, recurrent_dropout={{uniform(0, 1)}}))
    
    if ({{choice(['two', 'three'])}}) == 'two':
        model.add(Dense(512))
    
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer={{choice(['adam'])}} )
    model.fit(trainX, trainY, epochs=250, batch_size={{choice([1, 4, 16, 32, 64, 128])}},callbacks=callbacks,
                                                                                                  validation_split=0.25,)
    
    model.load_weights("forecasting_price_model.h5")
    
    # make predictions
    trainPredict = model.predict(trainX)
    testPredict = model.predict(testX)

    # calculate root mean squared error
    trainScore = math.sqrt(mean_squared_error(trainY[:], trainPredict[:,0]))
    print('Train Score: %.5f RMSE' % (trainScore))
    testScore = math.sqrt(mean_squared_error(testY[:], testPredict[:,0]))
    print('Test Score: %.5f RMSE' % (testScore))
    

    
    return {'loss': trainScore, 'status': STATUS_OK, 'model': model}

In [ ]:
best_run, best_model = optim.minimize(model=create_model,
                                              data=data,
                                              algo=tpe.suggest,
                                              max_evals=100,
                                              trials=Trials(),notebook_name='2. LSTM Network Optimization with Hyperas')

X_train, Y_train, X_test, Y_test = data()
print("Evalutation of best performing model:")
print(best_model.evaluate(X_test, Y_test))
print("Best performing model chosen hyper-parameters:")
print(best_run)
print(best_model.evaluate(X_train, Y_test))